# VacationPy
----

#### Google API

In [1]:
#!jupyter nbextension enable --py gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys_all import google_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# File to Load
cities_to_load = "cities.csv"

# Read School and Student Data File and store into Pandas Data Frames
cities_data = pd.read_csv(cities_to_load)
cities_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Horta,69,PT,1579770424,73,38.53,-28.63,53.186,3.85
1,1,Esperance,90,AU,1579813499,82,-33.87,121.90,62.600,3.60
2,2,Torbay,75,CA,1579777710,79,47.67,-52.73,26.996,6.20
3,3,Los Llanos de Aridane,59,ES,1579766634,63,28.66,-17.92,60.800,2.60
4,4,Busselton,2,AU,1579815105,51,-33.65,115.33,66.992,3.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key = google_api_key)

In [5]:
# Drop NA's
cities_data = cities_data.dropna()

# Store 'Lat' and 'Lng' into  locations 
locations = cities_data[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
Humidity = cities_data["Humidity"].astype(float)

In [6]:
# Configure Figure
figure_layout = {
    'width': '400px',
    'height': '600px',
    'margin': '0 auto 0 auto'
}

In [7]:
# Create a Heatmap layer
fig = gmaps.figure(layout = figure_layout) #, map_type="HYBRID")

heat_layer = gmaps.heatmap_layer(locations, weights = Humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', width='400px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# DataFrame with ideal weather condition
cities_data_ideal = cities_data.loc[(cities_data['Max Temp'] < 80) \
                                    & (cities_data['Max Temp'] > 70) \
                                    & (cities_data['Wind Speed'] < 10) \
                                    & (cities_data['Cloudiness'] == 0) \
                                    , :]
cities_data_ideal

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
25,25,Rikitea,0,PF,1579789972,79,-23.12,-134.97,79.304,7.82
44,44,Valdivia,0,CL,1579773060,47,-39.81,-73.25,75.200,4.10
54,54,Reconquista,0,AR,1579771223,59,-29.15,-59.65,77.558,2.19
141,141,Gwadar,0,PK,1579833386,54,25.12,62.33,71.132,3.14
241,241,Carnarvon,0,AU,1579816538,69,-24.87,113.63,75.200,4.60
281,281,Copiapo,0,CL,1579773992,47,-27.37,-70.33,77.000,6.20
494,494,Marsh Harbour,0,BS,1579780596,73,26.54,-77.06,71.366,8.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# New Varible
hotel_df = []

In [10]:
# Setting Coordinates and Some Parameters:
locations_ideal = cities_data_ideal[["Lat", "Lng"]].astype(float)
locations_ideal = locations_ideal.values.tolist()
ideal_cities = len(locations_ideal)
target_radius = 5000
target_type = "Hotel"

In [11]:
# Search for hotels around 5000 meters and calls to Google API
for C in range(ideal_cities):
    target_coordinates = str(locations_ideal[C][0]) + ", " + str(locations_ideal[C][1])
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": google_api_key
    }
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params = params).json()
    #print(json.dumps(response, indent = 4, sort_keys = True))
    hotel_df.append(response['results'][1]['name'])

In [12]:
cities_data_ideal['Hotel Name'] = hotel_df
cities_data_ideal.head()

/Applications/anaconda/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
25,25,Rikitea,0,PF,1579789972,79,-23.12,-134.97,79.304,7.82,Pension Maro'i
44,44,Valdivia,0,CL,1579773060,47,-39.81,-73.25,75.200,4.10,Hostal Entre Rios
54,54,Reconquista,0,AR,1579771223,59,-29.15,-59.65,77.558,2.19,Grand Hotel
141,141,Gwadar,0,PK,1579833386,54,25.12,62.33,71.132,3.14,National Bank of Pakistan (NBP)
241,241,Carnarvon,0,AU,1579816538,69,-24.87,113.63,75.200,4.60,Hospitality Carnarvon


In [13]:
narrowed_city_df = cities_data_ideal

In [14]:
hotel_df = cities_data_ideal

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig = gmaps.figure(layout = figure_layout)

fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', width='400px'))

Figure(layout=FigureLayout(height='600px', margin='0 auto 0 auto', width='400px'))